In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Read MCMC data from a .mcmc file. Remember that the file has a utf-8 encoding
def read_mcmc_data(file_path, encoding='utf-8'):
    data = pd.read_csv(file_path, delimiter='\t', encoding=encoding)
    return data

# MCMC dataset
file_path = "/Users/john/Downloads/HD177724.mcmc"

# Read the MCMC data from the .mcmc file with encoding
try:
    mcmc_data = read_mcmc_data(file_path, encoding='utf-8')
except IOError:
    print("Error: File not found or could not be read.")
    exit(1)

# A particular subset of columns for the pair plot (similar to Dr. Jones' correlation plot)
columns_of_interest = ['R_e', 'V_e', 'inc', 'T_p', 'pa']

# Select the subset of data for the columns of interest
subset_data = mcmc_data[columns_of_interest].dropna()

# Define pa_shift variable (True or False)
pa_shift = True

# Convert 'pa' values from degrees to radians if pa_shift is True
if pa_shift:
    subset_data['pa'] = np.radians(subset_data['pa'] % 180)

# Create the PairGrid with scatter plots on the lower and KDE plots on the upper triangle
g = sns.PairGrid(data=subset_data, diag_sharey=False)

# Plot scatter plots on the lower triangle
for i in range(len(columns_of_interest)):
    for j in range(len(columns_of_interest)):
        if i > j:
            g.axes[i, j].scatter(subset_data[columns_of_interest[j]], subset_data[columns_of_interest[i]], color='white', edgecolor='black', s=10)

# Map KDE plots in the upper and lower triangles with the 'rocket' color map
g.map_upper(sns.kdeplot, cmap='rocket', fill=True, levels=100)
g.map_lower(sns.kdeplot, cmap='rocket', fill=True, levels=100)

# Map histograms on the diagonal
g.map_diag(sns.histplot, kde=False, color="#03051A", bins=25)

# Set the background color of the pair grid
for ax in g.axes.flat:
    if isinstance(ax, plt.Axes) and ax.collections:
        ax.set_facecolor('#000000')  # Set black background color for pair grid

# Customizing x-axis tick positions and labels for 'pa' values
g.axes[-1, -1].set_xticks(np.linspace(0, np.pi, 5))  # 5 evenly spaced ticks from 0 to pi (180 degrees)
g.axes[-1, -1].set_xticklabels([f'{int(np.degrees(angle))}' for angle in np.linspace(0, np.pi, 5)])

# Color bar
cbar_ax = g.fig.add_axes([0.1, -0.1, 0.8, 0.03])  # Adjust the position and size of the color bar

# Create a color bar with the 'rocket' colormap
gradient = np.linspace(0, 1, 256).reshape(1, -1)
gradient = np.vstack((gradient, gradient))

# Plot the color bar with the 'rocket' colormap, otherwise, the code will plot in Viridis
cbar = plt.colorbar(mappable=plt.imshow(gradient, aspect='auto', cmap='rocket'),
                    cax=cbar_ax, orientation='horizontal')

# Color bar ticks
cbar_ax.tick_params(colors='black')

# Set the label for the color bar
cbar.set_label('Relative Density of Models', color='black') 
g.fig.suptitle("Pair Grid with Heat Maps and Histograms", y=1.02)

# Add best fit model CSV file
best_fit_data = pd.read_csv('/Users/john/Desktop/best_fitting_model.csv')

# Convert 'pa' values from degrees to radians in best_fit_data if pa_shift is True
if pa_shift:
    best_fit_data['pa'] = np.radians(best_fit_data['pa'] % 180)

# Plot the stars on top of the pair plot
for i in range(len(columns_of_interest)):
    for j in range(i):
        best_x = best_fit_data[columns_of_interest[j]].values[0]
        best_y = best_fit_data[columns_of_interest[i]].values[0]
        g.axes[i, j].scatter(best_x, best_y, marker='*', color='yellow', s=100, edgecolor='black', label='Best Fitting Model')

# Identify outliers and plot them as white filled circles (Dr. Jones asked to keep them)
outliers = subset_data.loc[subset_data['R_e'] > 5]  
for i in range(len(columns_of_interest)):
    for j in range(i):
        outlier_x = outliers[columns_of_interest[j]]
        outlier_y = outliers[columns_of_interest[i]]
        g.axes[i, j].scatter(outlier_x, outlier_y, marker='o', facecolors='white', edgecolor='black', s=100, alpha=0.5, label='Outliers')

# Create the legend
g.fig.legend(loc='center right', bbox_to_anchor=(1.15, 0.5))

# Adjust the left margin to add more space to the left
plt.subplots_adjust(left=0.15)

plt.savefig('heatmaps.png', dpi=500, bbox_inches='tight')
plt.show()
